In [23]:
import os

In [24]:
%pwd

'/Users/kjaggav1/Library/CloudStorage/OneDrive-JCPenney/myworkspace/practice/foodclassification'

In [25]:
#os.chdir("../")
%pwd

'/Users/kjaggav1/Library/CloudStorage/OneDrive-JCPenney/myworkspace/practice/foodclassification'

In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    object_key: Path
    local_data_file: Path
    unzip_dir: Path
    bucket_name: str
    access_key: str
    secret_access_key: str
    region: str

In [11]:

from src.foodclassification.constants import *
from src.foodclassification.utils.common import read_yaml, create_directories


In [12]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        cred = self.secret.aws_credentials
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            object_key=config.object_key,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            bucket_name=config.bucket_name,
            access_key=os.environ['AWS_ACCESS_KEY_ID'],
            secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
            region=os.environ['AWS_DEFAULT_REGION']
        )

        return data_ingestion_config

In [13]:
import os
import boto3
import yaml
import urllib.request as request
from box.exceptions import BoxValueError
import zipfile
from src.foodclassification import logger
from src.foodclassification.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            s3 = boto3.client(
                's3',
                aws_access_key_id=self.config.access_key,
                aws_secret_access_key=self.config.secret_access_key,
                region_name=self.config.region)

            bucket_name = self.config.bucket_name
            s3_file_key = self.config.object_key
            local_file_path = self.config.local_data_file

            try:
                s3.download_file(bucket_name, s3_file_key, local_file_path)
                logger.info(f"File '{s3_file_key}' downloaded to '{local_file_path}'.")
            except BoxValueError:
                raise ValueError("unable to download file from s3")
        else:
            logger.info(f"file already present of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)





In [31]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except BoxValueError:
    raise ValueError("unable to download file from s3")


[2024-09-06 13:13:30,181: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-06 13:13:30,185: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-06 13:13:30,186: INFO: common: yaml file: secrets/secrets.yaml loaded successfully]
[2024-09-06 13:13:30,187: INFO: common: created directory at: artifacts]
[2024-09-06 13:13:30,187: INFO: common: created directory at: artifacts/data_ingestion]
[2024-09-06 13:13:30,187: INFO: 752630733: file already present of size: ~ 771248 KB]
